In [26]:
import pandas as pd
data = pd.read_csv('case2.csv')
data.head()

,user id;event_time;event name;group;country;source
5,61645E+11;44928;click;test(6.99);Canada;
5,61952E+11;44928;click;test(6.99);Chile;
5,61989E+11;44928;click;test(6.99);Colombia;
5,60957E+11;44928;click;test(6.99);Italy;
5,6163E+11;44928;click;test(6.99);Mauritius;


Були проблеми з файлом у csv форматі, тому було прийнято рішення конвертувати в xlsx

In [27]:
data = pd.read_excel('case2.xlsx')
data.head()

,user id,event_time,event name,group,country,source
0,56164500000000000,44928,click,test(6.99),Canada,NaN
1,56195200000000000,44928,click,test(6.99),Chile,NaN
2,56198900000000000,44928,click,test(6.99),Colombia,NaN
3,56095700000000000,44928,click,test(6.99),Italy,NaN
4,5616300000000000,44928,click,test(6.99),Mauritius,NaN


In [28]:
def explore(x):
    divider = "*_*"
    print("\n {} \n".format((divider*20))) #creates a dvider between each method output breaking at each end.
    
    print("Dataframe Makeup \n") #title for output.
    
    x.info() # Explains what the data and values the data is madeup from.
    
    print("\n {} \n".format((divider*20))) #creates a dvider between each method output breaking at each end.
    
    print("Descriptive Statistics \n\n", x.describe().round(2)) #Gives a statstical breakdown of the data.
    
    print("\n {} \n".format((divider*20))) #creates a dvider between each method output breaking at each end.
    
    print("Shape of dataframe: {}".format(x.shape)) # Gives the shape of the data.
    
    print("\n {} \n".format((divider*20))) #creates a dvider between each method output breaking at each end.
    return
explore(data)


 *_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_* 

Dataframe Makeup 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96344 entries, 0 to 96343
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   user id     96344 non-null  object 
 1   event_time  96344 non-null  object 
 2   event name  96344 non-null  object 
 3   group       96344 non-null  object 
 4   country     96333 non-null  object 
 5   source      68557 non-null  float64
dtypes: float64(1), object(5)
memory usage: 4.4+ MB

 *_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_* 

Descriptive Statistics 

          source
count  68557.00
mean       3.68
std        2.03
min        1.00
25%        2.00
50%        4.00
75%        5.00
max       15.00

 *_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_* 

Shape of dataframe: (96344, 6)

 *_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_* 



In [29]:
data['source'] = data['source'].fillna(0)
data.head()

,user id,event_time,event name,group,country,source
0,56164500000000000,44928,click,test(6.99),Canada,0.0
1,56195200000000000,44928,click,test(6.99),Chile,0.0
2,56198900000000000,44928,click,test(6.99),Colombia,0.0
3,56095700000000000,44928,click,test(6.99),Italy,0.0
4,5616300000000000,44928,click,test(6.99),Mauritius,0.0


Переведення дати в звичний формат

In [39]:
data['event_time'] = pd.to_numeric(data['event_time'], errors='coerce')
data['event_time_converted'] = pd.to_datetime(data['event_time'], origin='1899-12-30', unit='D')
data.head()

,user id,event_time,event name,group,country,source,event_time_converted
0,56164500000000000,44928.0,click,test(6.99),Canada,0.0,2023-01-02
1,56195200000000000,44928.0,click,test(6.99),Chile,0.0,2023-01-02
2,56198900000000000,44928.0,click,test(6.99),Colombia,0.0,2023-01-02
3,56095700000000000,44928.0,click,test(6.99),Italy,0.0,2023-01-02
4,5616300000000000,44928.0,click,test(6.99),Mauritius,0.0,2023-01-02


In [30]:
df_group = data.groupby(['group', 'event name']).agg({'user id': 'nunique'}).reset_index()
df_group.head()

,group,event name,user id
0,control(9.99),click,8826
1,control(9.99),purchase,1499
2,test(6.99),click,8503
3,test(6.99),purchase,1608


In [31]:
df_group = data.groupby(['group', 'event name']).agg({'user id': 'count'}).reset_index()
df_group.head()

,group,event name,user id
0,control(9.99),click,45878
1,control(9.99),purchase,2639
2,test(6.99),click,44910
3,test(6.99),purchase,2917


In [32]:
clicks = data[data['event name'] == 'click'].groupby('group').size()
purchases = data[data['event name'] == 'purchase'].groupby('group').size()
results = pd.DataFrame({
    'clicks': clicks,
    'purchases': purchases
})
results.head()

,clicks,purchases
group,,
control(9.99),45878,2639
test(6.99),44910,2917


Підрахунки конверсії в покупку для обох планів

In [33]:
conversion_rates = purchases / clicks
results = pd.DataFrame({
    'clicks': clicks,
    'purchases': purchases,
    'conversion_rate': conversion_rates
})
results.head()

,clicks,purchases,conversion_rate
group,,,
control(9.99),45878,2639,0.057522
test(6.99),44910,2917,0.064952


Підрахунок revenue цих планів

In [34]:
prices = {'test(6.99)': 6.99, 'control(9.99)': 9.99}
revenue = purchases * pd.Series(prices)

In [35]:
results = pd.DataFrame({
    'clicks': clicks,
    'purchases': purchases,
    'conversion_rate': conversion_rates,
    'revenue': revenue
})
results.head()

,clicks,purchases,conversion_rate,revenue
control(9.99),45878,2639,0.057522,26363.61
test(6.99),44910,2917,0.064952,20389.83


In [36]:
#control group is more profitable despite having smaller CR

In [43]:
import numpy as np
from scipy import stats

# Кількість кліків і покупок в кожній групі
clicks_test = data[(data['event name'] == 'click') & (data['group'] == 'test(6.99)')].shape[0]
purchases_test = data[(data['event name'] == 'purchase') & (data['group'] == 'test(6.99)')].shape[0]
clicks_control = data[(data['event name'] == 'click') & (data['group'] == 'control(9.99)')].shape[0]
purchases_control = data[(data['event name'] == 'purchase') & (data['group'] == 'control(9.99)')].shape[0]

# Прибутки груп
revenue_test = purchases_test * 6.99
revenue_control = purchases_control * 9.99

# Розміри вибірок в групах
n_test = clicks_test
n_control = clicks_control

mean_revenue_test = revenue_test / purchases_test
mean_revenue_control = revenue_control / purchases_control

# T-тест для порівняння средніх значень прибутку
t_stat, p_val = stats.ttest_ind_from_stats(mean_revenue_test, np.sqrt(mean_revenue_test / n_test), purchases_test,
                                           mean_revenue_control, np.sqrt(mean_revenue_control / n_control), purchases_control)

# Виведення результатів
print(f'T-statistic: {t_stat:.2f}')
print(f'P-value: {p_val:.4f}')



T-statistic: -8206.80
P-value: 0.0000
